# Optimizing hyperparameters for DL models on the IDS2017 dataset

In this notebook, different DL models are used on the IDS2017 with hyperparameter optimization to test the performance. Deep neural networks, autoencoders, convolutional networks and RNNs are tested on the dataset.

In [2]:
from utils_ids2018 import load_ids2018, feature_selection
from notebook_utils import plot_confusion_matrix, metrics_report, calculate_metrics_by_label, test_metrics_DL, plot_overall_accuracy
from notebook_utils import test_metrics_AE
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import numpy as np
%matplotlib inline
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

attack_labels = {
    0: 'Benign',
    1: 'Bot',
    2: 'Brute Force -Web',
    3: 'Brute Force -XSS',
    4: 'DDOS attack-HOIC',
    5: 'DDOS attack-LOIC-UDP',
    6: 'DDoS attacks-LOIC-HTTP',
    7: 'DoS attacks-GoldenEye',
    8: 'DoS attacks-Hulk',
    9: 'DoS attacks-SlowHTTPTest',
    10: 'DoS attacks-Slowloris',
    11: 'FTP-BruteForce',
    12: 'Infilteration',
    13: 'SQL Injection',
    14: 'SSH-Bruteforce'
}

## Load Dataset

In [ ]:
df = load_ids2018()

In [ ]:
X = df.iloc[:, 0:78]
Y = df[["label", "is_attack", "label_code"]]

X.info()
Y.info()
print(Y.label.value_counts())

## Feature Selection

In [ ]:
X = feature_selection(X, Y)

## Split Dataset

The dataset is split into a training set and a testing set with a ratio of 0.8/0.2. The dataset is stratified according to the label to have an equal representation of all classes in the 2 subsets.

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y.label_code)

In [ ]:
Y_train.label.value_counts()

In [ ]:
Y_test.label.value_counts()

In [ ]:
benign_percentage = len(Y_train.label[Y_train["label"]=="BENIGN"])/len(Y_train)
print('Percentage of benign samples: %.4f' % benign_percentage)
print(Y_train.is_attack.value_counts())

## Smote Resampling

In [ ]:
from imblearn.over_sampling import SMOTE

def resample_dataset(X, Y, min_samples, attack_labels):
    Y = Y.drop(columns=['label'])
    combined = pd.concat([X, Y], axis=1)
    counts = Y['label_code'].value_counts()
    samples_number = {i: max(counts[i], min_samples) for i in np.unique(Y['label_code'])}
    combined_array = combined.values
    y_array = Y['label_code'].values
    resampler = SMOTE(random_state=42, sampling_strategy=samples_number)
    resampled_array, y_resampled = resampler.fit_resample(combined_array, y_array)
    X_resampled = resampled_array[:, :-Y.shape[1]]
    Y_resampled = resampled_array[:, -Y.shape[1]:]
    X_resampled_df = pd.DataFrame(X_resampled, columns=X.columns)
    Y_resampled_df = pd.DataFrame(Y_resampled, columns=Y.columns)
    Y_resampled_df['label'] = Y_resampled_df['label_code'].map(attack_labels)
    Y_resampled_df['label'] = Y_resampled_df['label'].astype('category')
    return X_resampled_df, Y_resampled_df

X_smote_train, Y_smote_train = resample_dataset(X_train, Y_train, 100000, attack_labels)


In [ ]:
Y_smote_train.label.value_counts()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler_smote = StandardScaler()
scaler_smote.fit(X_smote_train)

In [ ]:
# Save the model
def save_model(model, model_name):
    # Create directory if it does not exist
    model_dir = os.path.join("models", "DL_models_optimized_2018")
    os.makedirs(model_dir, exist_ok=True)
    # Save the model
    model.save(os.path.join(model_dir, f"{model_name}.keras"))

metrics = {}

## Optimized DNN

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define the combined model builder function
def build_combined_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1],)))

    # Choose the number of layers (either 3 or 4)
    num_layers = hp.Choice('num_layers', values=[3, 4])
    
    # Same dropout rate for all layers
    dropout_rate = hp.Choice('dropout', values=[0.0, 0.1, 0.2, 0.3, 0.4, 0.5])

    if num_layers == 3:
        # Fixed units for 3 layers: 128, 64, 32
        units_per_layer = [128, 64, 32]
    else:
        # Fixed units for 4 layers: 256, 128, 64, 32
        units_per_layer = [256, 128, 64, 32]
    
    for i in range(num_layers):
        model.add(Dense(units=units_per_layer[i], activation='relu'))
        model.add(Dropout(rate=dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    
    # Use Adam optimizer with different learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner for the combined model
tuner_combined = kt.Hyperband(build_combined_model,
                              objective='accuracy',
                              max_epochs=20,
                              factor=3,
                              directory='optimization_2018',
                              project_name='DNN_combined_fixed_dropout')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search for the combined model
tuner_combined.search(scaler_smote.transform(X_smote_train), Y_smote_train.is_attack, 
                      epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps_combined = tuner_combined.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps_combined.values}")

# Build and train the model with the optimal hyperparameters
model_combined = tuner_combined.hypermodel.build(best_hps_combined)
history_combined = model_combined.fit(scaler_smote.transform(X_smote_train), Y_smote_train.is_attack, 
                                      epochs=50, validation_split=0.2, callbacks=[stop_early])

# Evaluate and save the model
metrics["DNN_Optimized_Combined"] = test_metrics_DL("DNN_Optimized_Combined", model_combined, scaler_smote, X_test, Y_test, reshape=False)
save_model(model_combined, "DNN_SMOTE_Optimized_Combined")

## Optimized CNN

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Conv1D, Flatten, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

# Ensure TensorFlow compatibility
tf.compat.v1.reset_default_graph()

# Define the model builder function for CNN with fixed units and filters, and reduced dropout range
def build_cnn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1], 1)))
    
    # Fixed number of filters in the Conv1D layer
    model.add(Conv1D(filters=64,
                     kernel_size=hp.Int('kernel_size', min_value=2, max_value=5, step=1),
                     activation='relu'))
    
    # Flatten layer
    model.add(Flatten())
    
    # Fully connected layer with fixed units
    model.add(Dense(units=64, activation='relu'))
    
    # Dropout layer with reduced range
    model.add(Dropout(rate=hp.Choice('dropout', values=[0.0, 0.2, 0.4])))

    model.add(Dense(1, activation='sigmoid'))
    
    # Adam optimizer with different learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_cnn_model,
                     objective='accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='optimization_2018',
                     project_name='cnn_tuning_fixed_units_filters')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search
tuner.search(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
             Y_smote_train.is_attack, 
             epochs=50, 
             validation_split=0.2, 
             callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
                    Y_smote_train.is_attack, 
                    epochs=50, 
                    validation_split=0.2, 
                    callbacks=[stop_early])

# Evaluate and save the model
metrics["CNN_Optimized"] = test_metrics_DL("CNN_Optimized", model, scaler_smote, X_test, Y_test, reshape=False)
save_model(model, "CNN_SMOTE_Optimized")

## RNN Optimized

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Define the updated balanced model builder function for RNN
def build_updated_rnn_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_smote_train.shape[1], 1)))

    # Tune the number of units in the LSTM layer
    model.add(LSTM(units=hp.Int('units', min_value=64, max_value=128, step=32)))
    
    # Fully connected layer
    model.add(Dense(units=hp.Int('dense_units', min_value=32, max_value=64, step=32), activation='relu'))

    # Dropout layer with a range from 0 to 0.5
    model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.4, step=0.2)))

    # Output layer
    model.add(Dense(1, activation='sigmoid'))
    
    # Adam optimizer with the original set of learning rates
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_updated_rnn_model,
                     objective='accuracy',
                     max_epochs=20,
                     factor=3,
                     directory='optimization_2018',
                     project_name='updated_rnn_tuning')

# Early stopping callback
stop_early = EarlyStopping(monitor='accuracy', patience=5)

# Perform hyperparameter search
tuner.search(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
             Y_smote_train.is_attack, 
             epochs=50, 
             validation_split=0.2, 
             callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_smote.transform(X_smote_train).reshape(-1, X_smote_train.shape[1], 1), 
                    Y_smote_train.is_attack, 
                    epochs=50, 
                    validation_split=0.2, 
                    callbacks=[stop_early])

# Evaluate and save the model
metrics["RNN_Optimized"] = test_metrics_DL("RNN_Optimized", model, scaler_smote, X_test, Y_test, reshape=False)
save_model(model, "RNN_SMOTE_Optimized")


## Optimized Autoencoder

In [ ]:
import keras_tuner as kt
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

# Filter the training data to include only benign samples
benign_data = X_train[Y_train['is_attack'] == 0]
# Standardize the benign data
scaler_AE = StandardScaler()
scaler_AE.fit(benign_data)

# Define the model builder function for Autoencoder
def build_autoencoder_model_with_threshold(hp):
    model = Sequential()
    model.add(Input(shape=(benign_data.shape[1],)))

    # Encoder
    model.add(Dense(units=hp.Int('units_1', min_value=32, max_value=128, step=32), activation='relu'))
    model.add(Dense(units=hp.Int('bottleneck', min_value=8, max_value=16, step=8), activation='relu'))
    
    # Decoder
    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=32), activation='relu'))
    
    # Reconstruct the input
    model.add(Dense(benign_data.shape[1], activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4]))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Initialize the tuner
tuner = kt.Hyperband(build_autoencoder_model_with_threshold,
                     objective=kt.Objective("val_loss", direction="min"),
                     max_epochs=20,
                     factor=3,
                     directory='optimization_2018',
                     project_name='autoencoder_tuning_with_threshold')

# Early stopping callback
stop_early = EarlyStopping(monitor='val_loss', patience=5)

# Perform hyperparameter search, including threshold tuning
tuner.search(scaler_AE.transform(benign_data), scaler_AE.transform(benign_data), 
             epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"The optimal hyperparameters are: {best_hps.values}")

# Build the model with the optimal hyperparameters and train it
model = tuner.hypermodel.build(best_hps)
history = model.fit(scaler_AE.transform(benign_data), scaler_AE.transform(benign_data), 
                    epochs=50, validation_split=0.2, callbacks=[stop_early])



In [ ]:
# Get reconstruction errors on the validation set
reconstructions = model.predict(scaler_AE.transform(benign_data))
reconstruction_errors = np.mean(np.square(scaler_AE.transform(benign_data) - reconstructions), axis=1)

# Tune threshold by trying different percentiles
best_threshold = None
best_accuracy = -1
percentiles = range(80, 100)

for percentile in percentiles:
    threshold = np.percentile(reconstruction_errors, percentile)
    metrics["AE_Optimized"], metrics_by_label = test_metrics_AE(
        "AE_Optimized", model, scaler_AE, X_test, Y_test, threshold=threshold)

    # Assuming you have some F1 score metric in your `metrics` dictionary
    if metrics["AE_Optimized"]["accuracy"] > best_accuracy:
        best_accuracy = metrics["AE_Optimized"]["accuracy"]
        best_threshold = threshold

print(f"Optimal threshold: {best_threshold}, Best F1 Score: {best_f1_score}")

# Save the model
save_model(model, "AE_SMOTE_Optimized")

In [ ]:
# Define Autoencoder model
def create_autoencoder_model(input_shape):
    model = Sequential()
    model.add(Input(shape=(input_shape,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(input_shape, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
import keras_tuner as kt
from tensorflow.keras.optimizers import Adam

def build_autoencoder_model_with_fixed_layers(hp):
    model = AnomalyDetector(input_shape=benign_data.shape[1])
    
    # Compile the model with a tunable learning rate
    model.compile(optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='log')),
                  loss='mae')
    
    return model
# Instantiate the tuner for learning rate search
tuner = kt.RandomSearch(
    build_autoencoder_model_with_fixed_layers,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=2,
    directory='optimization_2018',
    project_name='autoencoder2'
)

# Perform the search
tuner.search(scaler_AE.transform(benign_data), scaler_AE.transform(benign_data),
             epochs=50, batch_size=512, validation_split=0.2,
             callbacks=[EarlyStopping(monitor='loss', patience=5)])

best_model = tuner.get_best_models(num_models=1)[0]

# Recalculate the threshold using the best model
reconstructions = best_model.predict(scaler_AE.transform(benign_data))
train_loss = np.mean(np.abs(scaler_AE.transform(benign_data) - reconstructions), axis=1)
threshold = np.mean(train_loss) + np.std(train_loss)
print("Optimal Learning Rate Threshold: ", threshold)

# Calculate the loss on validation data (you can use a part of benign_data for validation)
validation_loss = np.mean(np.abs(scaler_AE.transform(X_val) - best_model.predict(scaler_AE.transform(X_val))), axis=1)

# Test a range of thresholds
best_threshold = None
best_f1_score = 0
for t in np.linspace(np.min(validation_loss), np.max(validation_loss), 100):
    Y_pred = (validation_loss > t).astype(int)
    f1 = f1_score(Y_val['is_attack'], Y_pred)
    if f1 > best_f1_score:
        best_f1_score = f1
        best_threshold = t

print(f"Best Threshold: {best_threshold}, Best F1 Score: {best_f1_score}")


In [ ]:
metrics["AE"] = test_metrics_AE("Tuned AE", best_model, scaler_AE, X_test, Y_test, best_threshold)
save_model(best_model, "Tuned_AE_SMOTE")